In [1]:
import pandas as pd
import numpy as np
import requests


In [2]:
input_file_path = '/Users/thebekhruz/Desktop/100Days-Of-Code/100-Days-of-NLP-Odyssey/data/raw/sample_2k_swop_output_updated.jsonl'
df = pd.read_json(input_file_path, lines=True)
df.head(10)

,IAID,text,mentions,relations
0,a7bb9917-95ff-3f55-a640-4c5afcec25f2,View towards SE of junction of Queen Victoria ...,"[{'ne_span': 'Queen Victoria Road', 'ne_start'...",NaN
1,c29a7b77-7c46-3b85-88fe-05c8f4b2e384,"Front page of Bucks Free Press, Time capsule f...","[{'ne_span': 'Bucks Free Press', 'ne_start': 1...",NaN
2,196c11e6-f7b6-392f-ae41-28653345087c,"High Wycombe Police Station, in Queen Victoria...","[{'ne_span': 'High Wycombe Police Station', 'n...",NaN
3,7a5aace6-2398-3dcf-8843-37ff6ccea875,"Reference Library door, Queen Victoria Rd, Hig...","[{'ne_span': 'Reference Library', 'ne_start': ...",NaN
4,c66c4715-c03a-3aab-964b-e733f3ff1cf4,"Terrace of brick and flint cottages, Beech Rd,...","[{'ne_span': 'Beech Rd', 'ne_start': 37, 'ne_e...",NaN
5,d1159b13-8aa9-35c1-a4c2-fd13e24732b2,"Site of former Dial House, demolition at corne...","[{'ne_span': 'Dial House', 'ne_start': 15, 'ne...",NaN
6,e39a291d-ed39-3b56-b9aa-2022964a4114,"Row of cottages and part of factory building, ...","[{'ne_span': 'Coopers Yard', 'ne_start': 46, '...",[{'subject': 'https://www.wikidata.org/wiki/Q6...
7,3b84ea4c-e194-3c34-abbf-064a41ad59da,"60th anniversary of opening of Library, part o...","[{'ne_span': 'Library', 'ne_start': 31, 'ne_en...",NaN
8,bf418a27-d1d9-324b-a53f-50ab7ae8d81d,View towards NE of crossroads at junction of Q...,"[{'ne_span': 'Queen Victoria Rd', 'ne_start': ...",NaN
9,5bc83263-dcd0-3764-98cb-a3761480b4c7,"High Wycombe Police Station, Queen Victoria Ro...","[{'ne_span': 'High Wycombe Police Station', 'n...",NaN


In [3]:
# Delete relations column as it does not have any meaningful information
del df['relations']
df.head()

,IAID,text,mentions
0,a7bb9917-95ff-3f55-a640-4c5afcec25f2,View towards SE of junction of Queen Victoria ...,"[{'ne_span': 'Queen Victoria Road', 'ne_start'..."
1,c29a7b77-7c46-3b85-88fe-05c8f4b2e384,"Front page of Bucks Free Press, Time capsule f...","[{'ne_span': 'Bucks Free Press', 'ne_start': 1..."
2,196c11e6-f7b6-392f-ae41-28653345087c,"High Wycombe Police Station, in Queen Victoria...","[{'ne_span': 'High Wycombe Police Station', 'n..."
3,7a5aace6-2398-3dcf-8843-37ff6ccea875,"Reference Library door, Queen Victoria Rd, Hig...","[{'ne_span': 'Reference Library', 'ne_start': ..."
4,c66c4715-c03a-3aab-964b-e733f3ff1cf4,"Terrace of brick and flint cottages, Beech Rd,...","[{'ne_span': 'Beech Rd', 'ne_start': 37, 'ne_e..."


In [4]:
# Drop text column as it is not needed for the analysis
df = df.drop('text', axis=1)
df.head()

,IAID,mentions
0,a7bb9917-95ff-3f55-a640-4c5afcec25f2,"[{'ne_span': 'Queen Victoria Road', 'ne_start'..."
1,c29a7b77-7c46-3b85-88fe-05c8f4b2e384,"[{'ne_span': 'Bucks Free Press', 'ne_start': 1..."
2,196c11e6-f7b6-392f-ae41-28653345087c,"[{'ne_span': 'High Wycombe Police Station', 'n..."
3,7a5aace6-2398-3dcf-8843-37ff6ccea875,"[{'ne_span': 'Reference Library', 'ne_start': ..."
4,c66c4715-c03a-3aab-964b-e733f3ff1cf4,"[{'ne_span': 'Beech Rd', 'ne_start': 37, 'ne_e..."


In [5]:
# Explode the mentions column to get the individual mentions
df_exploded = df.explode('mentions')
df_exploded.head()

,IAID,mentions
0,a7bb9917-95ff-3f55-a640-4c5afcec25f2,"{'ne_span': 'Queen Victoria Road', 'ne_start':..."
0,a7bb9917-95ff-3f55-a640-4c5afcec25f2,"{'ne_span': 'High St', 'ne_start': 56, 'ne_end..."
0,a7bb9917-95ff-3f55-a640-4c5afcec25f2,"{'ne_span': 'Easton St', 'ne_start': 68, 'ne_e..."
0,a7bb9917-95ff-3f55-a640-4c5afcec25f2,"{'ne_span': 'High Wycombe', 'ne_start': 79, 'n..."
0,a7bb9917-95ff-3f55-a640-4c5afcec25f2,"{'ne_span': 'October 1936', 'ne_start': 93, 'n..."


In [6]:
# Convert dictionaries in 'mentions' column to separate columns
df_exploded = pd.concat([df_exploded.drop(['mentions'], axis=1), df_exploded['mentions'].apply(pd.Series)], axis=1)
df_exploded.head()

,IAID,ne_span,ne_start,ne_end,ne_type,id,0
0,a7bb9917-95ff-3f55-a640-4c5afcec25f2,Queen Victoria Road,31.0,50.0,LOC,NaN,NaN
0,a7bb9917-95ff-3f55-a640-4c5afcec25f2,High St,56.0,63.0,LOC,NaN,NaN
0,a7bb9917-95ff-3f55-a640-4c5afcec25f2,Easton St,68.0,77.0,LOC,NaN,NaN
0,a7bb9917-95ff-3f55-a640-4c5afcec25f2,High Wycombe,79.0,91.0,LOC,https://www.wikidata.org/wiki/Q64116,NaN
0,a7bb9917-95ff-3f55-a640-4c5afcec25f2,October 1936,93.0,105.0,DATE,NaN,NaN


In [7]:
# Drop the columns that are not needed for the analysis 
# which are 'ne_start', 'ne_end' and '0'
df_exploded = df_exploded.drop(['ne_start', 'ne_end', 0], axis=1)
df_exploded.head()

,IAID,ne_span,ne_type,id
0,a7bb9917-95ff-3f55-a640-4c5afcec25f2,Queen Victoria Road,LOC,NaN
0,a7bb9917-95ff-3f55-a640-4c5afcec25f2,High St,LOC,NaN
0,a7bb9917-95ff-3f55-a640-4c5afcec25f2,Easton St,LOC,NaN
0,a7bb9917-95ff-3f55-a640-4c5afcec25f2,High Wycombe,LOC,https://www.wikidata.org/wiki/Q64116
0,a7bb9917-95ff-3f55-a640-4c5afcec25f2,October 1936,DATE,NaN


In [8]:
# Rename 'id' column to 'wiki_page_id' to make it more meaningful
df_exploded = df_exploded.rename(columns={'id': 'wiki_page_id'})
df_exploded.head()


,IAID,ne_span,ne_type,wiki_page_id
0,a7bb9917-95ff-3f55-a640-4c5afcec25f2,Queen Victoria Road,LOC,NaN
0,a7bb9917-95ff-3f55-a640-4c5afcec25f2,High St,LOC,NaN
0,a7bb9917-95ff-3f55-a640-4c5afcec25f2,Easton St,LOC,NaN
0,a7bb9917-95ff-3f55-a640-4c5afcec25f2,High Wycombe,LOC,https://www.wikidata.org/wiki/Q64116
0,a7bb9917-95ff-3f55-a640-4c5afcec25f2,October 1936,DATE,NaN


In [22]:
# Count the number of unique IAID values
unique_iaid = df_exploded['IAID'].nunique()
print('Number of unique IAID values:', unique_iaid)

Number of unique IAID values: 2000


In [9]:
# Count how many ne_types  
df_exploded['ne_type'].value_counts()

ne_type
LOC     7621
ORG     2486
DATE    2117
PER     1843
MISC     659
Name: count, dtype: int64

In [18]:
# Filter rows where 'ne_type' is 'PER' and 'wiki_page_id' is not null
filtered_df = df_exploded[(df_exploded['ne_type'] == 'PER') & (df_exploded['wiki_page_id'].notnull())]

# Get the number of rows that satisfy the condition
num_rows = filtered_df.shape[0]

# Get total count for ne_type = 'PER'
total_count_PER = df_exploded[df_exploded['ne_type'] == 'PER'].shape[0]

# Print the number of rows
print("Number of rows where 'ne_type' is 'PER' and 'wiki_page_id' is not null:", num_rows)
print("Total count for ne_type = 'PER':", total_count_PER, "Difference in counts is: ", (total_count_PER - num_rows))


Number of rows where 'ne_type' is 'PER' and 'wiki_page_id' is not null: 257
Total count for ne_type = 'PER': 1843 Difference in counts is:  1586


In [20]:
# Filter rows where 'ne_type' is 'PER' and 'wiki_page_id' is not null
filtered_df = df_exploded[(df_exploded['ne_type'] == 'LOC') & (df_exploded['wiki_page_id'].notnull())]

# Get the number of rows that satisfy the condition
num_rows = filtered_df.shape[0]

# Get total count for ne_type = 'PER'
total_count_LOC = df_exploded[df_exploded['ne_type'] == 'LOC'].shape[0]

# Print the number of rows
print("Number of rows where 'ne_type' is 'LOC' and 'wiki_page_id' is not null:", num_rows)
print("Total count for ne_type = 'LOC':", total_count_LOC, "Difference in counts is: ", (total_count_LOC - num_rows))

Number of rows where 'ne_type' is 'LOC' and 'wiki_page_id' is not null: 2832
Total count for ne_type = 'LOC': 7621 Difference in counts is:  4789


In [10]:
# Filter out the rows where the wiki_page_id is null
df_filtered = df_exploded[df_exploded['wiki_page_id'].notnull()]
df_filtered.head()


,IAID,ne_span,ne_type,wiki_page_id
0,a7bb9917-95ff-3f55-a640-4c5afcec25f2,High Wycombe,LOC,https://www.wikidata.org/wiki/Q64116
0,a7bb9917-95ff-3f55-a640-4c5afcec25f2,High Wycombe,LOC,https://www.wikidata.org/wiki/Q64116
1,c29a7b77-7c46-3b85-88fe-05c8f4b2e384,Bucks Free Press,ORG,https://www.wikidata.org/wiki/Q4983472
1,c29a7b77-7c46-3b85-88fe-05c8f4b2e384,High Wycombe,LOC,https://www.wikidata.org/wiki/Q64116
2,196c11e6-f7b6-392f-ae41-28653345087c,High Wycombe,LOC,https://www.wikidata.org/wiki/Q64116


In [11]:
"""
- Exctract the id from wiki_page_id and drop the rest of the url
- Pattern to match Wikidata IDs starting with Q or P followed by one or more digits
    * [QP] matches a single character, either "Q" or "P".
    * (\d+) matches and captures one or more digits.
"""
pattern = r'[QP](\d+)'
# Use .loc to explicitly state that the operation should modify the 'wiki_page_id' column of df_filtered
# This makes it clear that the operation is intended to modify the DataFrame in place
df_filtered.loc[:, 'wiki_page_id'] = df_filtered['wiki_page_id'].str.extract(pattern)

df_filtered.head()

,IAID,ne_span,ne_type,wiki_page_id
0,a7bb9917-95ff-3f55-a640-4c5afcec25f2,High Wycombe,LOC,64116
0,a7bb9917-95ff-3f55-a640-4c5afcec25f2,High Wycombe,LOC,64116
1,c29a7b77-7c46-3b85-88fe-05c8f4b2e384,Bucks Free Press,ORG,4983472
1,c29a7b77-7c46-3b85-88fe-05c8f4b2e384,High Wycombe,LOC,64116
2,196c11e6-f7b6-392f-ae41-28653345087c,High Wycombe,LOC,64116


In [12]:
# Filter to only get ne_type PER
df_filtered_PER = df_filtered[df_filtered['ne_type'] == 'PER']
df_filtered_PER.head()

,IAID,ne_span,ne_type,wiki_page_id
35,fa0f621b-d0f6-3864-a0a5-5d4cda264d3f,Father Christmas,PER,1580236
50,12852cde-ff88-359a-b383-5264dbea862c,George VI,PER,280856
64,53a78c6e-5611-3689-ac3a-f2313e57d02a,Alfred Button,PER,21933817
120,383706ae-78c3-385f-8442-5cbd912e27fd,Elizabeth Barrett,PER,228494
134,bfa07f40-6ce2-364f-8664-bf469c366b13,Elizabeth Barrett,PER,228494


In [13]:
def query_wikidata_api(page_id):
    """
    Queries the Wikidata API for a given page ID and returns the raw JSON data.
    
    Parameters:
    - page_id (str): The Wikidata page ID to query.
    
    Returns:
    - dict: Raw JSON data from the Wikidata API.
    """
    url = f"https://www.wikidata.org/w/api.php?action=wbgetentities&ids=Q{page_id}&format=json&props=labels|descriptions|aliases|claims&languages=en"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return {}


In [14]:
def extract_data(raw_data):
    """
    Extracts required fields from the raw Wikidata API response.
    
    Parameters:
    - raw_data (dict): The raw JSON data from the Wikidata API.
    
    Returns:
    - dict: Extracted information including Label, Description, Also Known As, Instance Of, Sex or Gender, Part Of.
    """
    entity = list(raw_data['entities'].values())[0]
    label = entity['labels']['en']['value'] if 'en' in entity['labels'] else None
    description = entity['descriptions']['en']['value'] if 'en' in entity['descriptions'] else None
    aliases = [alias['value'] for alias in entity['aliases']['en']] if 'en' in entity['aliases'] else []
    instance_of_ids = [claim['mainsnak']['datavalue']['value']['id'] for claim in entity['claims']['P31']] if 'P31' in entity['claims'] else []
    sex_or_gender_ids = [claim['mainsnak']['datavalue']['value']['id'] for claim in entity['claims']['P21']] if 'P21' in entity['claims'] else []
    part_of_ids = [claim['mainsnak']['datavalue']['value']['id'] for claim in entity['claims']['P361']] if 'P361' in entity['claims'] else []
    
    # Use resolve_labels function to convert IDs to labels
    instance_of_labels = resolve_labels(instance_of_ids)
    sex_or_gender_labels = resolve_labels(sex_or_gender_ids)
    part_of_labels = resolve_labels(part_of_ids)
    
    return {
        'Label': label,
        'Description': description,
        'Also Known As': aliases,
        'Instance Of': instance_of_labels,
        'Sex or Gender': sex_or_gender_labels,
        'Part Of': part_of_labels
    }


In [15]:
def resolve_labels(ids):
    """
    Resolves a list of Wikidata property IDs to their corresponding labels.
    
    Parameters:
    - ids (list): A list of Wikidata property IDs.
    
    Returns:
    - list: A list of labels corresponding to the given IDs.
    """
    if not ids:
        return []
    ids_string = '|'.join(ids)
    url = f"https://www.wikidata.org/w/api.php?action=wbgetentities&ids={ids_string}&format=json&props=labels&languages=en"
    response = requests.get(url)
    labels = []
    if response.status_code == 200:
        data = response.json()
        for id in ids:
            label = data['entities'][id]['labels']['en']['value'] if 'en' in data['entities'][id]['labels'] else None
            if label:
                labels.append(label)
    return labels


In [16]:
def process_dataframe(df, page_id_column):
    """
    Updates the input DataFrame with new columns based on information extracted from Wikidata
    for each page ID in the specified column. Uses .loc to avoid SettingWithCopyWarning.
    
    Parameters:
    - df (pd.DataFrame): The input DataFrame containing Wikidata page IDs.
    - page_id_column (str): The name of the column containing the page IDs.
    
    Modifies:
    - The input DataFrame is updated in-place with new columns: 'Label', 'Description', 'Also Known As',
      'Instance Of', 'Sex or Gender', 'Part Of'.
    """
    # Initialize empty lists for each column to be added
    labels, descriptions, also_known_as, instances_of, sexes_or_genders, parts_of = [], [], [], [], [], []

    for page_id in df[page_id_column]:
        raw_data = query_wikidata_api(page_id)
        if raw_data:
            extracted_data = extract_data(raw_data)
        else:
            extracted_data = {'Label': None, 'Description': None, 'Also Known As': [], 'Instance Of': [], 'Gender': [], 'Part Of': []}
        
        # Append the data to the respective lists
        labels.append(extracted_data.get('Label'))
        descriptions.append(extracted_data.get('Description'))
        also_known_as.append(extracted_data.get('Also Known As'))
        instances_of.append(extracted_data.get('Instance Of'))
        sexes_or_genders.append(extracted_data.get('Sex or Gender'))
        parts_of.append(extracted_data.get('Part Of'))

    # Add the lists as new columns to the DataFrame using .loc to avoid warnings
    df.loc[:, 'Label'] = labels
    df.loc[:, 'Description'] = descriptions
    df.loc[:, 'Also Known As'] = also_known_as
    df.loc[:, 'Instance Of'] = instances_of
    df.loc[:, 'Gender'] = sexes_or_genders
    df.loc[:, 'Part Of'] = parts_of


In [17]:
# Sample the dataframe for the test.
# df_filtered_PER = df_filtered_PER.sample(10, random_state=42)
# df_filtered_PER.head()


In [18]:
# Assuming df is your DataFrame and 'page_id' is the column with Wikidata page IDs
process_dataframe(df_filtered_PER, 'wiki_page_id')
df_filtered_PER.head()


/var/folders/xh/x6c57wpx5hv7g4hxw7y4c6cc0000gn/T/ipykernel_25817/2548824889.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'Label'] = labels
/var/folders/xh/x6c57wpx5hv7g4hxw7y4c6cc0000gn/T/ipykernel_25817/2548824889.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'Description'] = descriptions
/var/folders/xh/x6c57wpx5hv7g4hxw7y4c6cc0000gn/T/ipykernel_25817/2548824889.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .

,IAID,ne_span,ne_type,wiki_page_id,Label,Description,Also Known As,Instance Of,Gender,Part Of
35,fa0f621b-d0f6-3864-a0a5-5d4cda264d3f,Father Christmas,PER,1580236,Father Christmas,Folkloric personification of Christmas in seve...,"[King Christmas, Sir Christmas, Old Chriſtmaſs...","[mythical character, folk saint, legendary fig...",[male],[]
50,12852cde-ff88-359a-b383-5264dbea862c,George VI,PER,280856,George VI,"King of the United Kingdom from 1936 to 1952, ...","[Albert Frederick Arthur George, King George V...",[human],[male],[George VI and Queen Elizabeth]
64,53a78c6e-5611-3689-ac3a-f2313e57d02a,Alfred Button,PER,21933817,Alfred Button,None,[],[human],[male],[]
120,383706ae-78c3-385f-8442-5cbd912e27fd,Elizabeth Barrett,PER,228494,Elizabeth Barrett Browning,"British poet, author","[Mrs. Browning, Elizabeth Barrett Barrett, Eli...",[human],[female],[]
134,bfa07f40-6ce2-364f-8664-bf469c366b13,Elizabeth Barrett,PER,228494,Elizabeth Barrett Browning,"British poet, author","[Mrs. Browning, Elizabeth Barrett Barrett, Eli...",[human],[female],[]


In [25]:
# Merge the data based on IAID 
df_merged = pd.merge(df_filtered, df_filtered_PER, how='inner', on='IAID')
df_merged.head()

,IAID,ne_span_x,ne_type_x,wiki_page_id_x,ne_span_y,ne_type_y,wiki_page_id_y,Label,Description,Also Known As,Instance Of,Gender,Part Of
0,fa0f621b-d0f6-3864-a0a5-5d4cda264d3f,High Wycombe,LOC,64116,Father Christmas,PER,1580236,Father Christmas,Folkloric personification of Christmas in seve...,"[King Christmas, Sir Christmas, Old Chriſtmaſs...","[mythical character, folk saint, legendary fig...",[male],[]
1,fa0f621b-d0f6-3864-a0a5-5d4cda264d3f,Father Christmas,PER,1580236,Father Christmas,PER,1580236,Father Christmas,Folkloric personification of Christmas in seve...,"[King Christmas, Sir Christmas, Old Chriſtmaſs...","[mythical character, folk saint, legendary fig...",[male],[]
2,12852cde-ff88-359a-b383-5264dbea862c,Coronation,MISC,5172236,George VI,PER,280856,George VI,"King of the United Kingdom from 1936 to 1952, ...","[Albert Frederick Arthur George, King George V...",[human],[male],[George VI and Queen Elizabeth]
3,12852cde-ff88-359a-b383-5264dbea862c,George VI,PER,280856,George VI,PER,280856,George VI,"King of the United Kingdom from 1936 to 1952, ...","[Albert Frederick Arthur George, King George V...",[human],[male],[George VI and Queen Elizabeth]
4,12852cde-ff88-359a-b383-5264dbea862c,High Wycombe,LOC,64116,George VI,PER,280856,George VI,"King of the United Kingdom from 1936 to 1952, ...","[Albert Frederick Arthur George, King George V...",[human],[male],[George VI and Queen Elizabeth]


Continue for 